In [ ]:
import sys
import copy
import math

BOARD_SIZE = 8

class Checkers:
    def __init__(self):
        """
        Initialize the checkers board and set the current player.
        Use 'r' and 'R' for red pieces and kings, 'b' and 'B' for black.
        Place 12 red and 12 black pieces on dark squares.
        """
        # 8x8 board init None
        self.board = [[None for i in range(BOARD_SIZE)] for j in range(BOARD_SIZE)]
        self.current_player = 'r'  # Red starts the game
        self.create_board()

    def create_board(self):
        """
        Set up the initial 8x8 board:
        - Red pieces on top 3 rows, black pieces on bottom 3 rows.
        - Only place pieces on dark squares ((row + col) % 2 == 0).
        """
        # Place Red in rows 0-2 on dark
        for row in range(3):
            for col in range(BOARD_SIZE):
                if (row + col) % 2 == 0:
                    self.board[row][col] = 'r'

        # Place Black in rows 5-7 on dark
        for row in range(5, BOARD_SIZE):
            for col in range(BOARD_SIZE):
                if (row + col) % 2 == 0:
                    self.board[row][col] = 'b'

    def print_board(self):
        """
        Display the current board with row and column numbers.
        """
        # Print column numbers at the top
        print("  ", end="") # dont move new line
        for col in range(BOARD_SIZE):
            print(f" {col} ", end="")
        print()

        # Print each row where . is empty
        for row in range(BOARD_SIZE):
            print(f"{row} ", end="")
            for col in range(BOARD_SIZE):
              if self.board[row][col]:
                piece = self.board[row][col]
              else:
                 piece = '.'
              print(f" {piece} ", end="")
            print(f" {row}")

        # Print column numbers at the bottom
        print("  ", end="")
        for col in range(BOARD_SIZE):
            print(f" {col} ", end="")
        print()

    def get_all_valid_moves(self, player):
        """
        Return a list of all valid moves for the given player ('r' or 'b').
        Include moves for regular and king pieces, and prioritize captures.
        """
        moves = []
        captures = []

        # Iterate through the board to find player's pieces
        for row in range(BOARD_SIZE):
            for col in range(BOARD_SIZE):
                piece = self.board[row][col] #'r', 'R', 'b', 'B', or None
                if piece and piece.lower() == player: # peice exists + belongs to current player
                    # Get moves for piece
                    piece_moves = self.get_valid_moves_for_piece(row, col)
                    for move in piece_moves:
                        # distant of 2 --> capture
                        if abs(move[0] - move[2]) == 2:
                            captures.append(move)
                        else:
                            moves.append(move)
        if captures:
          return captures
        else:
            return moves



    def get_valid_moves_for_piece(self, row, col):
        """
        Given a piece's position, return all valid move options for that piece.
        Account for direction (forward for regular, both for king) and captures.
        """
        moves = []
        piece = self.board[row][col]
        if not piece:
            return moves

        is_king = piece.isupper()
        is_red = piece.lower() == 'r'
        opponent = 'b' if is_red else 'r'

        # Determine movement directions based on piece type
        if is_king:
            # Kings can move in all four diagonal directions
            directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
        elif is_red:
            # Red regular pieces move downward (increasing row)
            directions = [(1, -1), (1, 1)]
        else:
            # Black regular pieces move upward (decreasing row)
            directions = [(-1, -1), (-1, 1)]

        # Check for simple moves (non-captures)
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < BOARD_SIZE and 0 <= new_col < BOARD_SIZE:
                if self.board[new_row][new_col] is None:
                    moves.append((row, col, new_row, new_col))

        # Check for captures (jump over opponent piece to empty square)
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            jump_row, jump_col = row + 2*dr, col + 2*dc
            if (0 <= new_row < BOARD_SIZE and 0 <= new_col < BOARD_SIZE and
                0 <= jump_row < BOARD_SIZE and 0 <= jump_col < BOARD_SIZE):
                if (self.board[new_row][new_col] and
                    self.board[new_row][new_col].lower() == opponent and
                    self.board[jump_row][jump_col] is None):
                    moves.append((row, col, jump_row, jump_col))

        return moves

    def make_move(self, move, player):
        """
        Apply the move to the board:
        - Move the piece
        - Remove any captured opponent piece
        - Promote to king if piece reaches the last row
        """
        from_row, from_col, to_row, to_col = move

        # ensure correct players peices
        if not self.board[from_row][from_col] or self.board[from_row][from_col].lower() != player:
            return False

        # Move piece
        piece = self.board[from_row][from_col]
        self.board[to_row][to_col] = piece
        self.board[from_row][from_col] = None

        # If capture remove captured piece
        if abs(to_row - from_row) == 2:
            captured_row = (from_row + to_row) // 2
            captured_col = (from_col + to_col) // 2
            self.board[captured_row][captured_col] = None # remove cap

        # Promote to king if reachef the last row
        if player == 'r' and to_row == BOARD_SIZE - 1:
            self.board[to_row][to_col] = 'R'
        elif player == 'b' and to_row == 0:
            self.board[to_row][to_col] = 'B'

        # Switch player
        if player == 'r':
           self.current_player = 'b'
        else:
          self.current_player = 'r'

        return True

    def is_game_over(self):
        """
        Check whether the game is over (no moves or no pieces for a player).
        """
        # Check if the current player has any valid moves
        moves = self.get_all_valid_moves(self.current_player)
        if not moves:
            winner = 'b' if self.current_player == 'r' else 'r'
            return True, winner

        # Count pieces for each player
        red_pieces = sum(1 for row in self.board for piece in row if piece and piece.lower() == 'r')
        black_pieces = sum(1 for row in self.board for piece in row if piece and piece.lower() == 'b')

        # If a player has no pieces, the other wins
        if red_pieces == 0:
            return True, 'b'
        if black_pieces == 0:
            return True, 'r'

        return False, None

    def evaluate(self):
        """
        Calculate board score: (1 × regular pieces) + (2 × kings).
        Return (AI's score - Opponent's score) to guide AI decisions.
        """
        red_score = 0
        black_score = 0

        # Compute scores for both players
        for row in range(BOARD_SIZE):
            for col in range(BOARD_SIZE):
                piece = self.board[row][col]
                if piece:
                    if piece == 'r':
                        red_score += 1
                    elif piece == 'R':
                        red_score += 2
                    elif piece == 'b':
                        black_score += 1
                    elif piece == 'B':
                        black_score += 2

        # AI is Black, so return Black's score minus Red's score
        return black_score - red_score

    def alphabeta(self, depth, alpha, beta, maximizing_player):
        """
        Use Minimax with Alpha-Beta Pruning to select the best move.
        Follows the pseudocode:
        - depth: Limits recursion to avoid infinite search
        - alpha: Best score maximizer can guarantee (initially -infinity)
        - beta: Best score minimizer can guarantee (initially +infinity)
        - maximizing_player: True for AI (Black), False for opponent (Red)
        """

        if depth == 0 or self.is_game_over()[0]: #terminal test
            # If game is over, return a large value based on winner
            game_over, winner = self.is_game_over()
            if game_over:
                if winner == 'b':
                    return 1000  # AI wins
                elif winner == 'r':
                    return -1000  # Opponent wins
            return self.evaluate()  # EVALUATION(node)

        if maximizing_player:
            # Maximizing player (AI, Black)
            value = -math.inf
            # Generate all possible moves
            for move in self.get_all_valid_moves('b'):
                # copy board to simulate the move
                board_copy = copy.deepcopy(self.board)
                current_player_copy = self.current_player
                self.make_move(move, 'b')

                # Recursively evaluate move (switch to minimizing)
                value = max(value, self.alphabeta(depth - 1, alpha, beta, False))

                # Restore board
                self.board = board_copy
                self.current_player = current_player_copy

                # Update alpha (best max)
                alpha = max(alpha, value)

                # if beta <= alpha, prune this branch
                if beta <= alpha:
                    break
            return value
        else:
            # Minimizing player (opponent, Red)
            value = math.inf
            # Generate all possible moves (MOVE_GEN)
            for move in self.get_all_valid_moves('r'):
                # copy board
                board_copy = copy.deepcopy(self.board)
                current_player_copy = self.current_player
                self.make_move(move, 'r')

                # Recursively evaluate the move (switch to maximizing)
                value = min(value, self.alphabeta(depth - 1, alpha, beta, True))

                # Restore board
                self.board = board_copy
                self.current_player = current_player_copy

                # Update beta (best min)
                beta = min(beta, value)

                # alpha cut
                # if beta <= alpha, prune this branch
                if beta <= alpha:
                    break
            return value

    def play_human_vs_ai(self):
        """
        Launch a human vs AI game of Checkers in the console.
        AI (Black) uses Alpha-Beta Pruning; human plays as Red.
        Display board, show legal moves, and accept input for human turns.
        """
        print("Welcome to Checkers! You are Red ('r'/'R'), AI is Black ('b'/'B').")
        depth = 3  # Depth limit for AI search

        while True:
            # Display the current board
            self.print_board()

            # Check if the game is over
            game_over, winner = self.is_game_over()
            if game_over:
                print(f"Game Over! Winner: {winner.upper()}")
                break

            if self.current_player == 'r':
                # Human's turn (Red)
                print("Your turn (Red).")
                moves = self.get_all_valid_moves('r')
                if not moves:
                    print("No valid moves! You lose.")
                    break

                # Show all possible moves
                for i, move in enumerate(moves):
                    print(f"{i}: {move}")

                # Get human input for move selection
                while True:
                    try:
                        choice = int(input("Enter move number: "))
                        if 0 <= choice < len(moves):
                            self.make_move(moves[choice], 'r')
                            break
                        else:
                            print("Invalid choice. Try again.")
                    except ValueError:
                        print("Enter a number. Try again.")
            else:
                # AI's turn (Black)
                print("AI's turn (Black)...")
                moves = self.get_all_valid_moves('b')
                if not moves:
                    print("AI has no valid moves! You win.")
                    break

                # Use Alpha-Beta Pruning to find the best move
                best_value = -math.inf
                best_move = None
                alpha = -math.inf
                beta = math.inf

                for move in moves:
                    board_copy = copy.deepcopy(self.board)
                    current_player_copy = self.current_player
                    self.make_move(move, 'b')

                    # Evaluate the move using Alpha-Beta Pruning
                    value = self.alphabeta(depth - 1, alpha, beta, False)

                    # Restore board state
                    self.board = board_copy
                    self.current_player = current_player_copy

                    # Update the best move if this move is better
                    if value > best_value:
                        best_value = value
                        best_move = move
                    alpha = max(alpha, value)

                # Apply the best move
                print(f"AI chooses move: {best_move}")
                self.make_move(best_move, 'b')

# Entry point
if __name__ == "__main__":
    game = Checkers()
    game.play_human_vs_ai()

Welcome to Checkers! You are Red ('r'/'R'), AI is Black ('b'/'B').
   0  1  2  3  4  5  6  7 
0  r  .  r  .  r  .  r  .  0
1  .  r  .  r  .  r  .  r  1
2  r  .  r  .  r  .  r  .  2
3  .  .  .  .  .  .  .  .  3
4  .  .  .  .  .  .  .  .  4
5  .  b  .  b  .  b  .  b  5
6  b  .  b  .  b  .  b  .  6
7  .  b  .  b  .  b  .  b  7
   0  1  2  3  4  5  6  7 
Your turn (Red).
0: (2, 0, 3, 1)
1: (2, 2, 3, 1)
2: (2, 2, 3, 3)
3: (2, 4, 3, 3)
4: (2, 4, 3, 5)
5: (2, 6, 3, 5)
6: (2, 6, 3, 7)
Enter move number: 3
   0  1  2  3  4  5  6  7 
0  r  .  r  .  r  .  r  .  0
1  .  r  .  r  .  r  .  r  1
2  r  .  r  .  .  .  r  .  2
3  .  .  .  r  .  .  .  .  3
4  .  .  .  .  .  .  .  .  4
5  .  b  .  b  .  b  .  b  5
6  b  .  b  .  b  .  b  .  6
7  .  b  .  b  .  b  .  b  7
   0  1  2  3  4  5  6  7 
AI's turn (Black)...
AI chooses move: (5, 1, 4, 0)
   0  1  2  3  4  5  6  7 
0  r  .  r  .  r  .  r  .  0
1  .  r  .  r  .  r  .  r  1
2  r  .  r  .  .  .  r  .  2
3  .  .  .  r  .  .  .  .  3
4  b  .  .  .  . 